# GermlineMasterWorkflow.wdl output check development notes

In [2]:
import os
import sys
import pandas as pd
import numpy as np
x_dir = '/Users/yo/zzIForge/fullyJan10/'
if not os.path.isdir(x_dir):
    print('directory not found\n', x_dir)

In [49]:
def get_rc_codes_df(x_dir):
    
    good_return_codes_list = ['0', '0\n']
    
    call_dirs = os.listdir(x_dir)
    call_dir_list = []
    call_dir_count = 0
    for cd in call_dirs:
        if os.path.isdir(os.path.join(cd,x_dir)) and cd[0:4] == 'call':
            call_dir_count += 1
            call_dir_list.append(cd)
    
    cols_list = ['rc', 'bam', 'bam.bai']
    rc_data = np.zeros((call_dir_count,len(cols_list))) -1
    rc_df = pd.DataFrame(rc_data, index=call_dir_list,columns=cols_list)
    
    root_trim_str, _ = os.path.split(x_dir)
    for dir_name, dir_list, files_list in os.walk(x_dir):
        for filename in files_list:
            full_filename = os.path.join(dir_name, filename)
            if filename in cols_list:
                if filename == 'rc':

                    with open(full_filename, 'r') as fh:
                        lines = fh.readlines()

                    if lines[0] in good_return_codes_list:
                        for call_dir in call_dir_list:
                            if call_dir in dir_name:
                                top_dir = dir_name.replace(root_trim_str, '..')
                                rc_df.loc[call_dir, 'rc'] = str(lines[0]).strip()

            else:
                for call_dir in call_dir_list:
                    if call_dir in dir_name:
                        fname, fext = os.path.splitext(filename)
                        some_stuff = os.stat(full_filename)
                        if fext[1:] == 'bam':
                            rc_df.loc[call_dir, 'bam'] = str(some_stuff.st_size)
                        elif fext[1:] == 'bai':
                            rc_df.loc[call_dir, 'bam.bai'] = str(some_stuff.st_size)
                                
    return rc_df                 

In [50]:
somedf = get_rc_codes_df(x_dir)
somedf

,rc,bam,bam.bai
call-DHVC,0,-1,-1
call-realign,0,65349394,1431360
call-align,0,59500466,1431328
call-bqsr,0,67462520,1431360
call-dedup,0,65322724,1431360
call-haplotype,0,67462520,1431360
call-merge,0,59500466,1431328
call-vqsr,0,-1,-1
call-trimseq,0,-1,-1
call-DAB,0,65349394,1431360


In [6]:
call_dirs = os.listdir(x_dir)
call_dir_list = []
call_dir_count = 0
for cd in call_dirs:
    if os.path.isdir(os.path.join(cd,x_dir)) and cd[0:4] == 'call':
        call_dir_count += 1
        call_dir_list.append(cd)
print(call_dir_count)

10


In [8]:
np.zeros(call_dir_count)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [11]:
cols_list = ['rc',]
rc_df = pd.DataFrame(np.zeros((call_dir_count,len(cols_list))), index=call_dir_list,columns=cols_list)
rc_df

,rc
call-DHVC,0.0
call-realign,0.0
call-align,0.0
call-bqsr,0.0
call-dedup,0.0
call-haplotype,0.0
call-merge,0.0
call-vqsr,0.0
call-trimseq,0.0
call-DAB,0.0
